In [3]:
%cd /content/drive/MyDrive/Agriculture App/agriculture-predictor-planner

/content/drive/MyDrive/Agriculture App/agriculture-predictor-planner


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import joblib

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
df = pd.read_csv('data/your_dataset.csv')

# --- Step 2: Preprocess the Data ---

# Convert 'Dist Name' to a categorical variable and then to a numeric code.
df['Dist Name'] = df['Dist Name'].astype('category')
df['District_Code'] = df['Dist Name'].cat.codes

# Convert the 'Month' column into a numeric month (1-12).
# Assuming the Month column contains month names (e.g., 'January', 'February', etc.)
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
df['Month_Num'] = df['Month'].apply(lambda x: month_order.index(x) + 1 if x in month_order else np.nan)

# Define the input features for the model.
features = ['District_Code', 'Month_Num']

# --- Step 3: Train a Model for Each Crop ---

# List the crops you want to train models for.
crops = ['RICE', 'WHEAT', 'MAIZE', 'PEARL MILLET', 'CHICKPEA', 'GROUNDNUT', 'SUGARCANE']

for crop in crops:
    target = f"{crop} YIELD (Kg per ha)"

    # Check if the target column exists in the dataframe.
    if target in df.columns:
        # Select the necessary columns and drop rows with missing values.
        df_crop = df[features + [target]].dropna()
        X = df_crop[features]
        y = df_crop[target]

        # Split the data into training and testing sets.
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Initialize and train the XGBoost regressor.
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        model.fit(X_train, y_train)

        # Evaluate the model.
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f"{crop} Model RMSE: {rmse:.2f}")

        # Save the trained model to disk.
        model_filename = f"model_{crop}.pkl"
        joblib.dump(model, model_filename)
        print(f"Saved {crop} model as {model_filename}")
    else:
        print(f"Target column {target} not found in dataset.")


In [6]:
#Load the data
df = pd.read_csv("data/merged/crop_soil_weather_merged.csv")

In [8]:
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7841 entries, 0 to 7840
Data columns (total 163 columns):
 #    Column                                                  Dtype  
---   ------                                                  -----  
 0    Dist Code                                               int64  
 1    Year                                                    int64  
 2    State Code                                              int64  
 3    State Name                                              object 
 4    Dist Name                                               object 
 5    RICE AREA (1000 ha)                                     float64
 6    RICE PRODUCTION (1000 tons)                             float64
 7    RICE YIELD (Kg per ha)                                  float64
 8    PEARL MILLET AREA (1000 ha)                             float64
 9    PEARL MILLET PRODUCTION (1000 tons)                     float64
 10   PEARL MILLET YIELD (Kg per ha)                

In [ ]:
# Convert 'Dist Name' to a categorical variable and 'Dito a numeric code.
df['Dist Name'] = df['Dist Name'].astype('category')
df['State Name'] = df['State Name'].astype('category')
